# 4. Run SMBalance Module

# 4.1 Import Functions

In [1]:
import os
import glob
#import WAPORWA modules
os.chdir(r'C:\WA_Souss_Massa_Training\WAPOR\modules') #change to modules path
import WA
from WA.pickle_basin import pickle_in,pickle_out  
from WA.model_SMBalance import run_SMBalance
from dask.distributed import Client

#Read pickle
Main_dir=r"C:\WA_Souss_Massa_Training\WAPOR\Souss_Massa\Main"
pickle=glob.glob(os.path.join(Main_dir,'*.pickle'))[-1] 
BASIN=pickle_in(pickle)  

#Customize dask performance
client = Client()
client

C:\Anaconda3\envs\waporwa_wks\lib\site-packages\distributed\node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 64158 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://127.0.0.1:64159 Dashboard: http://127.0.0.1:64158/status,Cluster Workers: 4 Cores: 8 Memory: 8.39 GB


In [2]:
BASIN

{'Name': 'Souss_massa',
 'Dir': 'C:\\WA_Souss_Massa_Training\\WAPOR\\Souss_Massa\\Main',
 'time_range': ['2012-01-01', '2013-12-31'],
 'end_month': 'DEC',
 'geo_data': {'basin': 'C:\\WA_Souss_Massa_Training\\WAPOR\\Souss_Massa\\shapefile\\sous_massa_wgs.shp'},
 'global_data': {'grace': 'C:\\WA_Souss_Massa_Training\\WAPOR\\Souss_Massa\\Global\\GRACE\\GSFC.glb.200301_201607_v02.4-ICE6G',
  'grand': 'C:\\WA_Souss_Massa_Training\\WAPOR\\Souss_Massa\\Global\\GRanD\\GRanD_reservoirs_v1_3.shp',
  'wdpa': 'C:\\WA_Souss_Massa_Training\\WAPOR\\Souss_Massa\\Global\\WDPA\\WDPA_CatIandII_17countries.shp'},
 'input_data': {'yearly': {'lcc': ['C:\\WA_Souss_Massa_Training\\WAPOR\\Souss_Massa\\Input\\L2_LCC_A',
    '-',
    'Landcover Class'],
   'lu': ['C:\\WA_Souss_Massa_Training\\WAPOR\\Souss_Massa\\Main\\data\\luwa',
    '-',
    'WA+ Landuse Categories']},
  'monthly': {'p': ['C:\\WA_Souss_Massa_Training\\WAPOR\\Souss_Massa\\Input\\L1_PCP_M',
    'mm/month',
    'Precipitation'],
   'et': ['C:\\WA

# 4.2 Run SM Balance

In [3]:
import  datetime
starttime = datetime.datetime.now()


## Read model input data from pickled dictionary
MAIN_FOLDER= os.path.join(BASIN['Dir'],'data','nc')


## read netCDF files    
smsat_file = BASIN['main_data']['stat']['thetasat'] #saturated soil moisture content
p_in = BASIN['main_data']['monthly']['p'] # monthly precipitation
e_in = BASIN['main_data']['monthly']['et'] # monthly actual evapotranspiration
i_in = BASIN['main_data']['monthly']['i'] # monthly interception
rd_in = BASIN['main_data']['monthly']['nRD'] # monthly number of rainy days
lu_in = BASIN['main_data']['yearly']['lcc']  # monthly land cover class

outputs=run_SMBalance(MAIN_FOLDER,
                      p_in, e_in, i_in, rd_in, lu_in, smsat_file,                      
                      f_perc=1, # percolation factor
                      f_Smax=0.9, # threshold percolation parameter
                      cf =  20, # f_Ssat soil mositure correction factor to componsate the variation in filling up and drying in a month
                      chunks=[1,300,300] # chunk size [time,lon, lat]
                     )

BASIN['main_data']['monthly']['etrain']=outputs[0]
BASIN['main_data']['monthly']['etincr']=outputs[1]
pickle_out(BASIN)



endtime = datetime.datetime.now()
print(endtime-starttime)

<xarray.DataArray 'Top-soil Saturated Water Content' (latitude: 1965, longitude: 2737)>
dask.array<xarray-<this-array>, shape=(1965, 2737), dtype=float32, chunksize=(300, 300), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 31.11 31.11 31.11 31.11 ... 29.17 29.17 29.17
  * longitude  (longitude) float32 -10.18 -10.18 -10.18 ... -7.469 -7.468 -7.467
Attributes:
    units:     %
    quantity:  Top-soil Saturated Water Content
    source:    WaPOR
    period:    stat
time:  0
<xarray.DataArray 'Precipitation' (latitude: 1965, longitude: 2737)>
dask.array<where, shape=(1965, 2737), dtype=float32, chunksize=(300, 300), chunktype=numpy.ndarray>
Coordinates:
    time       datetime64[ns] 2012-01-01
  * latitude   (latitude) float32 31.11 31.11 31.11 31.11 ... 29.17 29.17 29.17
  * longitude  (longitude) float32 -10.18 -10.18 -10.18 ... -7.469 -7.468 -7.467
time:  1
<xarray.DataArray 'Precipitation' (latitude: 1965, longitude: 2737)>
dask.array<where, shape=(1965, 2737

<xarray.DataArray 'Precipitation' (latitude: 1965, longitude: 2737)>
dask.array<where, shape=(1965, 2737), dtype=float32, chunksize=(300, 300), chunktype=numpy.ndarray>
Coordinates:
    time       datetime64[ns] 2013-09-01
  * latitude   (latitude) float32 31.11 31.11 31.11 31.11 ... 29.17 29.17 29.17
  * longitude  (longitude) float32 -10.18 -10.18 -10.18 ... -7.469 -7.468 -7.467
time:  21
<xarray.DataArray 'Precipitation' (latitude: 1965, longitude: 2737)>
dask.array<where, shape=(1965, 2737), dtype=float32, chunksize=(300, 300), chunktype=numpy.ndarray>
Coordinates:
    time       datetime64[ns] 2013-10-01
  * latitude   (latitude) float32 31.11 31.11 31.11 31.11 ... 29.17 29.17 29.17
  * longitude  (longitude) float32 -10.18 -10.18 -10.18 ... -7.469 -7.468 -7.467
time:  22
<xarray.DataArray 'Precipitation' (latitude: 1965, longitude: 2737)>
dask.array<where, shape=(1965, 2737), dtype=float32, chunksize=(300, 300), chunktype=numpy.ndarray>
Coordinates:
    time       datetime64[ns] 

In [4]:
endtime = datetime.datetime.now()
print(endtime-starttime)

0:08:56.163128


In [5]:
#43min